In [1]:
!pip install gTTS pydub gradio deep-translator langdetect
!apt install ffmpeg -y

Defaulting to user installation because normal site-packages is not writeable


'apt' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
from gtts import gTTS
from pydub import AudioSegment
from langdetect import detect
from deep_translator import GoogleTranslator
import gradio as gr
import tempfile
import io
import textwrap

C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Emotion Modifier

In [3]:
def modify_audio(audio_segment, emotion):
    if emotion == "Happy":
        return audio_segment.speedup(playback_speed=1.3).apply_gain(4)
    elif emotion == "Sad":
        return audio_segment.speedup(playback_speed=0.8).apply_gain(-3)
    elif emotion == "Angry":
        return audio_segment.speedup(playback_speed=1.4).apply_gain(5)
    elif emotion == "Calm":
        return audio_segment.speedup(playback_speed=0.9).apply_gain(0)
    return audio_segment  # Neutral

Language Detection + Translation

In [4]:
def translate_to_marathi(text):
    try:
        lang = detect(text)
        if lang != 'mr':
            translated = GoogleTranslator(source='auto', target='mr').translate(text)
            return translated
        else:
            return text  # already Marathi
    except:
        return text  # fallback, read as-is


TTS Generator with Long Text Support

In [5]:
def text_to_speech(text, emotion, auto_translate):
    # Translate if enabled
    if auto_translate:
        text = translate_to_marathi(text)

    # Split long text (~200 chars) for gTTS
    chunks = textwrap.wrap(text, width=200)
    combined_audio = AudioSegment.empty()

    for chunk in chunks:
        try:
            tts = gTTS(text=chunk, lang='mr')  # Marathi pronunciation of any script
        except:
            tts = gTTS(text=chunk)  # fallback

        buf = io.BytesIO()
        tts.write_to_fp(buf)
        buf.seek(0)
        audio = AudioSegment.from_file(buf, format="mp3")
        combined_audio += audio

    final_audio = modify_audio(combined_audio, emotion)

    # Save to temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
        final_audio.export(tmp_file.name, format="mp3")
        return tmp_file.name

Gradio interface

In [6]:
gr.Interface(
    fn=text_to_speech,
    inputs=[
        gr.Textbox(label="Enter Any Language Text"),
        gr.Dropdown(["Neutral", "Happy", "Sad", "Angry", "Calm"], label="Emotion", value="Neutral"),
        gr.Checkbox(label="Auto-translate to Marathi (Optional)", value=True)
    ],
    outputs=gr.Audio(label="Generated Voice", type="filepath"),
    title="🌍 Multilingual Text-to-Speech (Hindi | English | Marathi)",
    description="Supports emotion, long text, and optional translation to Marathi. Mixed words also supported."
).launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
Traceback (most recent call last):
  File "C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prajw\AppData\Roaming\Python\Python312

Keyboard interruption in main thread... closing server.
